## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-19-14-52-53 +0000,wapo,Foundations step in to offer $37 million lifel...,https://www.washingtonpost.com/business/2025/0...
1,2025-08-19-14-52-20 +0000,bbc,Why Ukraine's first lady wrote Melania Trump a...,https://www.bbc.com/news/articles/czd08nj08r3o...
2,2025-08-19-14-51-52 +0000,nyt,Live Updates: Trump Says U.S. Won’t Send Peace...,https://www.nytimes.com/live/2025/08/19/us/tru...
3,2025-08-19-14-51-49 +0000,bbc,What to know as Air Canada says it will resume...,https://www.bbc.com/news/articles/cq680gm5q86o...
4,2025-08-19-14-46-44 +0000,bbc,Starmer hails breakthrough on Ukraine - but an...,https://www.bbc.com/news/articles/cr5rl6y04z0o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,48
7,ukraine,28
262,zelensky,22
21,canada,15
52,putin,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
242,2025-08-18-18-02-57 +0000,nypost,Trump ‘very happy’ after Ukraine peace summit ...,https://nypost.com/2025/08/18/us-news/trump-ze...,149
138,2025-08-19-02-43-00 +0000,wsj,Trump Pushes for Putin and Zelensky Meeting in...,https://www.wsj.com/world/trump-pushes-for-pea...,136
20,2025-08-19-14-07-00 +0000,wsj,"After months of effort, President Trump says a...",https://www.wsj.com/world/trump-plans-to-get-p...,122
74,2025-08-19-10-25-29 +0000,nypost,Putin and Zelensky could meet within next two ...,https://nypost.com/2025/08/19/world-news/putin...,115
271,2025-08-18-16-05-04 +0000,nyt,Trump Will Likely Press Zelensky to Make Conce...,https://www.nytimes.com/2025/08/18/us/zelensky...,112


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
242,149,2025-08-18-18-02-57 +0000,nypost,Trump ‘very happy’ after Ukraine peace summit ...,https://nypost.com/2025/08/18/us-news/trump-ze...
88,70,2025-08-19-09-44-56 +0000,cbc,"Air Canada, flight attendants reach tentative ...",https://www.cbc.ca/news/business/air-canada-co...
221,58,2025-08-18-19-54-53 +0000,nypost,Putin claims Ukraine should give up Donetsk be...,https://nypost.com/2025/08/18/world-news/putin...
167,55,2025-08-19-00-01-00 +0000,wsj,The Trump administration has restored a websit...,https://www.wsj.com/politics/policy/lawmakers-...
11,48,2025-08-19-14-28-34 +0000,nyt,Netanyahu Faces Pressure From Far Right Over N...,https://www.nytimes.com/2025/08/19/world/middl...
284,38,2025-08-18-15-18-49 +0000,nypost,Marvel leaves Georgia to film superhero movies...,https://nypost.com/2025/08/18/business/marvel-...
286,34,2025-08-18-15-03-21 +0000,wapo,Newsmax to pay $67 million to settle Dominion ...,https://www.washingtonpost.com/business/2025/0...
140,34,2025-08-19-02-27-43 +0000,nypost,Elon Musk baby mama Ashley St. Clair reveals n...,https://nypost.com/2025/08/18/us-news/elon-mus...
72,33,2025-08-19-10-36-51 +0000,nypost,Chris Pratt calls RFK Jr. ‘wonderful’ and wish...,https://nypost.com/2025/08/19/us-news/chris-pr...
275,30,2025-08-18-15-52-15 +0000,cbc,Palestinians leave Gaza City fearing Israeli o...,https://www.cbc.ca/news/world/gaza-city-palest...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
